## Usage Note - comment out analysis not currently in use or it takes forever to run!

In [1]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import colorsys
import os

# Exact pixel values

In [2]:
# Define a list of image filenames
image_files = ['love-5s-barcode.png', 'family-5s-barcode.png', 'death-5s-barcode.png', 'conflict-5s-barcode.png']

In [3]:
# Loop through each image file
for file in image_files:

    # Load the image file
    image = plt.imread(file)

    # Convert the image to a numpy array
    image_array = np.array(image)

    # Convert the RGB values to the 0-255 range
    image_array = np.round(image_array * 255).astype(np.uint8)

    # Reshape the array to a list of RGB values
    pixel_list = image_array.reshape((image_array.shape[0] * image_array.shape[1]), 3)

    # Count the frequency of each colour
    colour_count = {}

    for pixel in pixel_list:
        colour = tuple(pixel)
        if colour in colour_count:
            colour_count[colour] += 1
        else:
            colour_count[colour] = 1            

    # Sort the colours by frequency
    sorted_colours = sorted(colour_count.items(), key=lambda x: x[1], reverse = True)[:9]

    palette = []

    # Print the top colours for this image
    print(f'The most dominant/common colours in {file} are:')
    for colour in sorted_colours:
        print(f'- {colour[0]} (count: {colour[1]})')
        palette.append(colour[0])

    # Create a plot with a single row and the number of columns equal to the length of the palette
    fig, ax = plt.subplots(figsize=(8, 1), ncols=len(palette), gridspec_kw={"wspace":0})

    # Iterate over each color in the palette and plot a color block
    for i, color in enumerate(palette):
        ax[i].imshow([[color]])
        ax[i].axis('off')

    plt.show()

# K-Means (9 clusters, RGB)

In [4]:
# Loop over the file names
for file in image_files:
    # Load the image file
    image = Image.open(file)

    # Convert the image to a numpy array
    image_array = np.array(image)

    # Flatten the array to a list of RGB values
    pixel_list = image_array.reshape((-1, 3))

    # Perform K-Means clustering with 9 clusters
    kmeans = KMeans(n_clusters = 9, random_state = 42).fit(pixel_list)

    # Get the representative colors of each cluster
    palette = kmeans.cluster_centers_.astype(int)

    # Print the RGB values of the palette
    print(f"Palette for {file}:")
    print(palette)

    # Plot the color blocks of the palette
    fig, ax = plt.subplots()
    ax.imshow(np.array([palette]), aspect='auto')
    ax.axis('off')
    # Save the palette as a PNG file
    fig.savefig(f"{file.split('.')[0]}-palette.png")
    plt.close(fig)

# K-Means (9 clusters, HSV)

In [5]:
# Loop over the file names
for file in image_files:
    
    # Load the image file
    image = Image.open(file)

    # Convert the image to a numpy array
    image_array = np.array(image)

    # Flatten the array to a list of RGB values
    pixel_list = image_array.reshape((-1, 3))

    # Perform K-Means clustering with 9 clusters
    kmeans = KMeans(n_clusters = 9, random_state = 42).fit(pixel_list)

    # Get the representative colors of each cluster
    palette_rgb = kmeans.cluster_centers_.astype(int)

    # Convert RGB values to HSV values
    palette_hsv = [colorsys.rgb_to_hsv(r / 255.0, g / 255.0, b / 255.0) for r, g, b in palette_rgb]

    # Print the HSV values of the palette
    print(f"Palette for {file}:")
    print(palette_hsv)

    # Plot the color blocks of the palette
    fig, ax = plt.subplots()
    ax.imshow(np.array([palette_rgb]), aspect='auto')
    ax.axis('off')
    plt.show()

# Colour Histograms

In [2]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
def compare_colour_hist(img1, img2):
    # Convert images to HSV color space
    hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)
    
    # Calculate the color histograms for both images
    hist1 = cv2.calcHist([hsv1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([hsv2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    
    # Normalize the histograms
    cv2.normalize(hist1, hist1)
    cv2.normalize(hist2, hist2)
    
    # Calculate the Bhattacharyya coefficient between the histograms
    bc = cv2.compareHist(hist1, hist2, cv2.HISTCMP_BHATTACHARYYA)
    
    return bc

# Similarity Scores

In [8]:
# Read barcode PNG images
love = cv2.imread('love-5s-barcode.png', cv2.IMREAD_UNCHANGED)
family = cv2.imread('family-5s-barcode.png', cv2.IMREAD_UNCHANGED)
death = cv2.imread('death-5s-barcode.png', cv2.IMREAD_UNCHANGED)
conflict = cv2.imread('conflict-5s-barcode.png', cv2.IMREAD_UNCHANGED)

# Loop over all combinations of the images and store similarity scores in a list
barcode_images = [(love, "love"), (family, "family"), (death, "death"), (conflict, "conflict")]

similarity_scores = []

for i in range(len(barcode_images)):
    for j in range(i+1, len(barcode_images)):
        
        # Calculate similarity score using colour histogram method
        similarity_score = compare_colour_hist(barcode_images[i][0], barcode_images[j][0])
        similarity_scores.append((f"{barcode_images[i][1]} and {barcode_images[j][1]}", similarity_score))

# Sort the similarity scores in decreasing order and print them
similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

for score in similarity_scores_sorted:
    print(f"Similarity between {score[0]}: {score[1]:.4f}")
    
df = pd.DataFrame(similarity_scores_sorted, columns=['Image Pair', 'Similarity Score'])

# Write the DataFrame to a CSV file
df.to_csv('barcode_similarity_scores.csv', index=False)

Similarity between love and conflict: 0.4976
Similarity between love and death: 0.4835
Similarity between love and family: 0.4797
Similarity between death and conflict: 0.4658
Similarity between family and death: 0.4549
Similarity between family and conflict: 0.3907


In [9]:
# Create a matrix of similarity scores
barcode_similarity_matrix = np.zeros((4, 4))
for i in range(len(barcode_images)):
    for j in range(i+1, len(barcode_images)):
        barcode_similarity_matrix[i][j] = similarity_scores_sorted[(4*i + j - (i+1)*(i+2)//2)][1]
        barcode_similarity_matrix[j][i] = barcode_similarity_matrix[i][j]

# Create a list of labels for the x and y axis of the heatmap
labels = [img[1] for img in barcode_images]

# Create a heatmap of the similarity scores
fig, ax = plt.subplots()
im = ax.imshow(barcode_similarity_matrix, cmap='coolwarm', vmin=0, vmax=1)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

# Rotate the tick labels and set their alignment
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add the score value to each cell of the heatmap
for i in range(len(labels)):
    for j in range(len(labels)):
        text = ax.text(j, i, f"{barcode_similarity_matrix[i][j]:.2f}", ha="center", va="center", color="black")

# Add a colorbar to the heatmap
cbar = ax.figure.colorbar(im, ax=ax)

# Set the title of the heatmap
ax.set_title("Similarity Scores of Barcode Images")

# Show the heatmap
plt.show()

In [10]:
# Read palette PNG images
love = cv2.imread('love-5s-barcode-palette.png', cv2.IMREAD_UNCHANGED)
family = cv2.imread('family-5s-barcode-palette.png', cv2.IMREAD_UNCHANGED)
death = cv2.imread('death-5s-barcode-palette.png', cv2.IMREAD_UNCHANGED)
conflict = cv2.imread('conflict-5s-barcode-palette.png', cv2.IMREAD_UNCHANGED)

# Loop over all combinations of the images and store similarity scores in a list
palette_images = [(love, "love"), (family, "family"), (death, "death"), (conflict, "conflict")]

similarity_scores = []

for i in range(len(palette_images)):
    for j in range(i+1, len(palette_images)):
        
        # Calculate similarity score using colour histogram method
        similarity_score = compare_colour_hist(palette_images[i][0], palette_images[j][0])
        similarity_scores.append((f"{palette_images[i][1]} and {palette_images[j][1]}", similarity_score))

# Sort the similarity scores in decreasing order and print them
similarity_scores_sorted = sorted(similarity_scores, key=lambda x: x[1], reverse=True)

for score in similarity_scores_sorted:
    print(f"Similarity between {score[0]}: {score[1]:.4f}")

In [11]:
# Create a matrix of similarity scores
palette_similarity_matrix = np.zeros((4, 4))
for i in range(len(palette_images)):
    for j in range(i+1, len(palette_images)):
        palette_similarity_matrix[i][j] = similarity_scores_sorted[(4*i + j - (i+1)*(i+2)//2)][1]
        palette_similarity_matrix[j][i] = palette_similarity_matrix[i][j]

# Create a list of labels for the x and y axis of the heatmap
labels = [img[1] for img in palette_images]

# Create a heatmap of the similarity scores
fig, ax = plt.subplots()
im = ax.imshow(palette_similarity_matrix, cmap='coolwarm', vmin=0, vmax=1)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(labels)))
ax.set_yticks(np.arange(len(labels)))
ax.set_xticklabels(labels)
ax.set_yticklabels(labels)

# Rotate the tick labels and set their alignment
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

# Add the score value to each cell of the heatmap
for i in range(len(labels)):
    for j in range(len(labels)):
        text = ax.text(j, i, f"{palette_similarity_matrix[i][j]:.2f}", ha="center", va="center", color="black")

# Add a colorbar to the heatmap
cbar = ax.figure.colorbar(im, ax=ax)

# Set the title of the heatmap
ax.set_title("Similarity Scores of Palette Images")

# Show the heatmap
plt.show()

# Frame-wise colour analysis

In [4]:
# Path to the directory containing the image files
folder_path = "./Paradise frames/"

# Loop over the file names
for file_name in os.listdir(folder_path):
    # Load the image file
    file_path = os.path.join(folder_path, file_name)
    image = Image.open(file_path)

    # Convert the image to a numpy array
    image_array = np.array(image)

    # Flatten the array to a list of RGB values
    pixel_list = image_array.reshape((-1, 3))

    # Perform K-Means clustering with 3 clusters
    kmeans = KMeans(n_clusters = 3, random_state = 42).fit(pixel_list)

    # Get the representative colors of each cluster
    palette = kmeans.cluster_centers_.astype(int)

    # Print the RGB values of the palette
    print(f"Palette for {file_name}:")
    print(palette)

    # Plot the color blocks of the palette
    fig, ax = plt.subplots()
    ax.imshow(np.array([palette]), aspect='auto')
    ax.axis('off')

    # Save the palette as a PNG file
    palette_file_name = f"palette-{os.path.splitext(file_name)[0]}.png"
    palette_file_path = os.path.join(folder_path, palette_file_name)
    fig.savefig(palette_file_path)
    plt.close(fig)


C:\Users\Maddie\AppData\Local\Temp\ipykernel_15652\3449134691.py:17: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters = 3, random_state = 42).fit(pixel_list)


Palette for frame0.png:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
Palette for frame1.png:
[[ 71 187 210]
 [157 242 223]
 [230 229 199]]
Palette for frame10.png:
[[ 21  46  57]
 [123 186 210]
 [ 97 100 104]]
Palette for frame100.png:
[[ 60  50  57]
 [ 16   8  10]
 [130 128 135]]
Palette for frame101.png:
[[ 14   6   8]
 [184 154 166]
 [ 99  66  75]]
Palette for frame102.png:
[[208  87  54]
 [ 53  24  32]
 [234 165 118]]
Palette for frame103.png:
[[ 88  82  87]
 [209 178 171]
 [ 30  21  27]]
Palette for frame104.png:
[[  4   3   3]
 [108  86  81]
 [193 183 183]]
Palette for frame105.png:
[[  2   1   1]
 [195 184 183]
 [115  85  83]]
Palette for frame106.png:
[[  5   4   4]
 [232 230 232]
 [155 144 147]]
Palette for frame107.png:
[[126 132 145]
 [ 21  21  26]
 [211 216 225]]
Palette for frame108.png:
[[108 117 127]
 [157 171 180]
 [ 15  15  17]]
Palette for frame109.png:
[[120  62  61]
 [164 209 175]
 [ 79 157 108]]
Palette for frame11.png:
[[106 144 126]
 [212 229 227]
 [ 63 112  94]]
Palette for fram

C:\Users\Maddie\AppData\Local\Temp\ipykernel_15652\3449134691.py:17: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  kmeans = KMeans(n_clusters = 3, random_state = 42).fit(pixel_list)


Palette for frame28.png:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
Palette for frame29.png:
[[152  33  31]
 [ 36   8  85]
 [232 228 235]]
Palette for frame3.png:
[[248 228 156]
 [228 100 243]
 [105  82  65]]
Palette for frame30.png:
[[214 225 203]
 [218  95  57]
 [ 23   6  46]]
Palette for frame31.png:
[[ 46  10  78]
 [245 202 122]
 [210  64  60]]
Palette for frame32.png:
[[ 25   5  49]
 [228  97 116]
 [ 62  10 174]]
Palette for frame33.png:
[[ 36   9  80]
 [239 165  43]
 [169 151 225]]
Palette for frame34.png:
[[227  69  67]
 [233 145 150]
 [ 33  88  83]]
Palette for frame35.png:
[[ 12 106  64]
 [229   1   6]
 [237 142 171]]
Palette for frame36.png:
[[148 114 173]
 [ 73  29 124]
 [166 131 190]]
Palette for frame37.png:
[[ 75 181 106]
 [235 194 165]
 [228  65  94]]
Palette for frame38.png:
[[222  86 110]
 [116 169  90]
 [ 63  48  81]]
Palette for frame39.png:
[[ 17  11  20]
 [248 185 210]
 [200 129  75]]
Palette for frame4.png:
[[228  99 243]
 [249 228 155]
 [107  82  64]]
Palette for frame40.png:
[